In [251]:
from transformers import AutoTokenizer, AutoModel
import urllib.request
import pandas as pd
from tqdm import tqdm

In [3]:
def load_data():

    urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
    urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

    train_data = pd.read_table("ratings_train.txt")
    test_data = pd.read_table('ratings_test.txt')

    train_data = train_data.dropna(how = 'any')
    train_data = train_data.reset_index(drop=True)

    test_data = test_data.dropna(how = 'any')
    test_data = test_data.reset_index(drop=True)

    return train_data, test_data

In [8]:
train_data, test_data = load_data()

In [241]:
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2", bos_token = "<s>", eos_token = "</s>", pad_token = "<pad>")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [246]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, data_labels = [],[]

    for example, label in tqdm(zip(examples,labels), total = len(examples)):

        bos_token = tokenizer.bos_token
        eos_token = tokenizer.eos_token

        tokens = bos_token + example + eos_token
        input_id = tokenizer(tokens, padding="max_length", max_length = max_seq_len, truncation = True, return_tensors="pt")
        input_ids.append(input_id)
        data_labels.append(label)
    

    return input_ids, data_labels


        

In [249]:
train_X, train_y = convert_examples_to_features(train_data["document"], train_data["label"], max_seq_len = 128, tokenizer = tokenizer)
test_X, test_y = convert_examples_to_features(test_data["document"], test_data["label"], max_seq_len = 128, tokenizer = tokenizer)

100%|██████████| 49997/49997 [00:19<00:00, 2579.30it/s]


In [252]:
model = AutoModel.from_pretrained("skt/kogpt2-base-v2")

Downloading: 100%|██████████| 513M/513M [00:09<00:00, 53.2MB/s] 
Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [256]:
output = model(**train_X[0])

In [378]:
output["past_key_values"]

((tensor([[[[-9.0507e-01,  7.3533e-01, -5.0451e-01,  ..., -6.4107e-01,
             -5.1330e-01, -5.8635e-01],
            [-3.3533e-01,  2.3255e+00, -1.8136e+00,  ..., -1.3133e+00,
             -1.4428e+00, -7.6552e-01],
            [ 2.0960e-01,  2.6828e+00, -2.2977e+00,  ..., -1.6052e+00,
             -1.5029e+00, -8.1309e-02],
            ...,
            [-8.0174e-01,  4.2414e-01,  9.5840e-01,  ...,  2.3361e-01,
              7.7483e-01,  8.5740e-01],
            [-8.4418e-01,  4.4112e-01,  9.3640e-01,  ...,  2.1761e-01,
              7.5902e-01,  8.2010e-01],
            [-8.6976e-01,  4.0597e-01,  9.4882e-01,  ...,  2.2730e-01,
              7.5720e-01,  7.9777e-01]],
  
           [[-6.6549e-02,  8.0200e-01, -1.2728e-01,  ..., -1.0774e+00,
              1.0303e+00, -3.3901e-01],
            [-8.1676e-02,  6.6009e-01,  2.4095e+00,  ..., -8.0750e-01,
             -1.8701e+00,  1.1264e+00],
            [-1.3665e-01,  7.1082e-01,  2.6588e+00,  ..., -1.2902e-01,
             -2.3342